In [25]:
import tensorflow as tf
import numpy as np
import pandas as pd

In [26]:
train_data = pd.read_csv('/root/autodl-tmp/wq/RS_Keras/data/Sohu2022_data/rec_data/train-dataset.csv')
test_data = pd.read_csv('/root/autodl-tmp/wq/RS_Keras/data/Sohu2022_data/rec_data/test-dataset.csv') 

In [27]:
dict_cols = [x for x in train_data.columns if x not in ['sampleId','label','pvId','userSeq','logTs',]]

In [28]:
names = globals()

## Step 1 对id类特征和类别特征进行映射

####  逻辑：
1.先对已有数据做映射
2.添加新数据只需要改动映射表，即此时如果新来一个id就在最大值后开辟新空间加1

##### 1.基础数据唯一值、字典映射、字典大小

In [29]:
from tqdm import tqdm
for col in tqdm(dict_cols):
    #基础数据
    names[str(col)+'_values' ] = list(train_data[col].unique())
    #基础映射表,维护
    names[str(col)+'_dict' ] = {value:idx for idx,value in enumerate(names[str(col)+'_values' ])}
    #字典大小
    names[str(col)+'_dict_size'] = len(names[str(col)+'_dict' ])

100%|██████████| 8/8 [00:00<00:00, 19.43it/s]


##### 2.在基础数据前提下增加新数据

In [31]:
for col in tqdm(dict_cols):
    for value in tqdm(test_data[col].values):
        if value in names[str(col)+'_dict' ]:
            continue
        else:
            names[str(col)+'_dict_size']= names[str(col)+'_dict_size'] +1
            names[str(col)+'_values' ].append(value)
            names[str(col)+'_dict' ][value] = names[str(col)+'_dict_size']
        

100%|██████████| 8/8 [00:09<00:00,  1.23s/it]


In [22]:
## 数据有['suv','itemId','operator','browserType','deviceType','osType','province','city']的映射字典

## Step 2 构建统计特征


In [77]:
#找到热门item
hot_item_value_counts = train_data.itemId.value_counts().sort_values(ascending=False).iloc[:100]
hot_item = list(hot_item_value_counts.index)

In [87]:
#找到活跃用户
hot_suv_value_counts = train_data.suv.value_counts().sort_values(ascending=False).iloc[:100]
hot_suv = list(hot_suv_value_counts.index)

In [79]:
train_data.head()

,sampleId,label,pvId,suv,itemId,userSeq,logTs,operator,browserType,deviceType,osType,province,city
0,1,0,1420332726637uyIZR2O,1581173072343wnwm3q,10092752,10094375:1640918979673;10084835:1640836934361;...,1641039890894,2203920,1583865,2326891,3585570,1518261,3058140
1,2,0,1420332726637uyIZR2O,1581173072343wnwm3q,10085565,10094375:1640918979673;10084835:1640836934361;...,1641039890894,2203920,1583865,2326891,3585570,1518261,3058140
2,3,0,1420332726637uyIZR2O,1581173072343wnwm3q,10105937,10094375:1640918979673;10084835:1640836934361;...,1641039890894,2203920,1583865,2326891,3585570,1518261,3058140
3,4,0,1420332726637uyIZR2O,1581173072343wnwm3q,10082274,10094375:1640918979673;10084835:1640836934361;...,1641039890894,2203920,1583865,2326891,3585570,1518261,3058140
4,5,0,1420332726637uyIZR2O,1581173072343wnwm3q,10083446,10094375:1640918979673;10084835:1640836934361;...,1641039890894,2203920,1583865,2326891,3585570,1518261,3058140


In [ ]:
#计算用户点击率：要不要以时间刻度